## Imports

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import random
import unicodedata
import string
import torch
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from torch import nn, optim
from SearchClassifier.word_classifier.data_loader import random_training_example, split_dataset, load_dataset, \
    split_dataset_reduced
from SearchClassifier.word_classifier.mutiLayerRNN import MultilayerRnn
from SearchClassifier.word_classifier.predict import evaluate, predict
from SearchClassifier.word_classifier.rnn import RNN
from SearchClassifier.word_classifier.test import test_with_plot
from SearchClassifier.word_classifier.train import train, train_loop
from SearchClassifier.word_classifier.utils import unicode_to_ascii, letter_to_tensor, line_to_tensor, n_letters, \
    category_from_output, load_checkpoint, save_checkpoint

## Global variables
Change for running configuration

In [3]:
!ls

drive  sample_data  SearchClassifier


In [4]:
TRAINING_PERCENT = 70
TEST_PERCENT = 20
VIEW_PERCENT = 10
LEARNING_RATE = 0.00005  # If you set this too high, it might explode. If too low, it might not learn
EPOCHS = 70000
PRINT_EVERY = 500
PLOT_EVERY = 500
N_HIDDEN = 1500
N_HIDDEN_2 = 512
EXEC_MODE = 1
REDUCE = 300
PATH_MODEL_1 = "drive/My Drive/PracticasEmpresa/models/model.pt"
PATH_MODEL_2 = "drive/My Drive/PracticasEmpresa/models/model2.pt"
PATH_DATASET = "drive/My Drive/PracticasEmpresa/data/kn8_defined_cleaned_data.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0))

cuda:0
Tesla P100-PCIE-16GB


### Configuration
Data loading and environment configuration

In [5]:
dataset = load_dataset(PATH_DATASET)

   kn8                       product
0    0              DNA-HSA-Konjugat
1    0  DNA Ultrasound-treated ELISA
2    0                   EARMASTER 7
3    0                    E-LICENCER
4    0                     FG-BOMBER


In [6]:
print(dataset.keys())

dict_keys([0, 1011010, 1011090, 1012100, 1012910, 1012990, 1013000, 1019000, 1021090, 1022190, 1022905, 1022910, 1022949, 1022959, 1023910, 1029069, 1039110, 1041010, 1041080, 1051920, 1061900, 2023090, 2031110, 2031211, 2031219, 2031915, 2031955, 2032110, 2032211, 2032219, 2032290, 2032911, 2032913, 2032915, 2032955, 2032990, 2061010, 2062200, 2062999, 2064100, 2064900, 2071410, 2071420, 2071430, 2071440, 2071460, 2071491, 2071499, 2072590, 2072699, 2072710, 2072740, 2072791, 2072799, 2081010, 2081090, 2089030, 2090011, 2091011, 2101219, 2101290, 2101981, 2101990, 2102010, 2102090, 2109959, 2109990, 3019190, 3019918, 3019919, 3031200, 3033200, 3037430, 3037955, 3037998, 3042013, 3042029, 3042055, 3042056, 3042058, 3042075, 3042083, 3042085, 3042091, 3042094, 3042903, 3042913, 3042919, 3042921, 3042929, 3042931, 3042933, 3042955, 3042956, 3042958, 3042985, 3042991, 3042999, 3046100, 3046200, 3047110, 3047190, 3047200, 3047300, 3047411, 3047415, 3047419, 3047500, 3047950, 3048100, 30483

In [7]:
training_dataset, test_dataset, view_dataset = split_dataset_reduced(dataset, TRAINING_PERCENT, TEST_PERCENT, VIEW_PERCENT, REDUCE)
all_categories = list(training_dataset.keys())
print(len(all_categories))

54


## Creating RNN

In [8]:
if EXEC_MODE == 1:
    rnn = MultilayerRnn(n_letters, N_HIDDEN, len(all_categories))
    rnn.to(device)
elif EXEC_MODE == 2:
    rnn = RNN(n_letters, N_HIDDEN, len(all_categories))
    rnn2 = MultilayerRnn(n_letters, N_HIDDEN, len(all_categories))
else:
    print("Loading network")
    rnn = RNN(n_letters, N_HIDDEN, len(all_categories))
    rnn = load_checkpoint(rnn, PATH_MODEL_1)
    #rnn.eval()
inputChar = letter_to_tensor('A').to(device)

hidden = torch.zeros(1, N_HIDDEN).to(device)

rnn

MultilayerRnn(
  (i2h): Linear(in_features=1557, out_features=1500, bias=True)
  (i2o): Linear(in_features=1557, out_features=54, bias=True)
  (softmax): LogSoftmax()
)

### Getting output

In [9]:
output, next_hidden = rnn(inputChar, hidden)
category_from_output(all_categories, output)

(8039010, 0)

### Showing samples

In [10]:
for i in range(10):
    category, line, category_tensor, line_tensor = random_training_example(all_categories, training_dataset)
    print('category =', category, '/ line =', line)

category = 90279050 / line = Lamp Holder G4 for Humalyzer 3500
category = 92099920 / line = Teile und Zubeh├╖r fΓü┐r Klarinetten, Trompeten, Dudels╬úcke, Mundharmonikas und andere Musik-Blasinstrumenten der Pos. 9205 >> 1 Stck BG L15 Ligature Baritone Sax Blattschraube (Pos. 016.00) 99/1000
category = 94018000 / line = FIRST CLASS plus BR Cosmos Black ZA / PKW Sicherheitskindersitz
category = 92099920 / line = Teile und Zubeh├╖r fΓü┐r Klarinetten, Trompeten, Dudels╬úcke, Mundharmonikas und andere Musik-Blasinstrumenten der Pos. 9205 >> 1 Stck Jody Jazz Power ring HRA1 Minus (Pos. 004.00)
category = 90139090 / line = Teile und Zubeh├╖r fΓü┐r in Kapitel 90 anderweit weder genannte noch inbegriffene optische Instrumente, Apparate und Ger╬úte sowie Teile und Zubeh├╖r von Lasern, ausg. Laserdioden HIER Scan-K├╖pfe (Markierung von Metall) SCANcube 10 SN: 395649
category = 87149110 / line = 53596 21 / 29 Fahrradrahmen aus Aluminium, lackiert,
category = 92099400 / line = Teile und Zubeh├╖r fΓ

### Defining criterion
NLLLoss() is good because the last layer of the RNN is nn.LogSoftmax I have to see the different criterion and activation functions

In [0]:
if EXEC_MODE == 1 or EXEC_MODE == 2:
    criterion = nn.NLLLoss()

## Training
each loop of training
  1) Create input and target tensors
  2) Create zeroed initial hidden state
  3) Read each letter in and keep hidden state for next letter
  4) Compare final output to target
  5) Back-propagate
  6) Return to output and loss
Keep track of losses for plotting

In [0]:
if EXEC_MODE == 1:
    print("Training network")
    rnn, all_losses = train_loop(rnn, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)
if EXEC_MODE == 2:
    print("Training first network")
    rnn, all_losses = train_loop(rnn, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)
    print("Training second network")
    rnn2, all_losses2 = train_loop(rnn2, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)

Training network


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


500 0% (0m 16s) 3.9563 Schellen fΓü┐r Halterung Sicherheitsnetz Trampolin / 85389099 ✗ (73269098)
1000 1% (0m 34s) 4.0675 543660 EW BELT CLIP / 87149110 ✗ (85189000)
1500 2% (0m 54s) 4.0147 1 StΓü┐ck 10' Geometry Messmolch s/n P08/10G001/5/3.C mit Zubeh├╖r, Nichtelektronische Instrumente, Apparate, Ger╬úte und Maschinen zum Messen oder PrΓü┐fen geometrischer Gr├╖ΓûÇen / 85389099 ✗ (90318091)
2000 2% (1m 13s) 4.0105 15 St Rainbow Folienabdeckung fΓü┐r Displaytasten, aus Kunststofffolien (Polyester) hergestellt, fΓü┐r Rainbow Rollenfarbwechsler / 73269098 ✗ (39269092)
2500 3% (1m 33s) 3.9687 Abtauuhr, TemperaturfΓü┐hler, Schnappverschluss / 39269097 ✗ (84189990)
3000 4% (1m 53s) 3.9794 13 St. MA Lichtsteuerger╬úte (keine speicherprogrammierbaren Steuerungen, keine numerische Steuerung mit eingebauter Datenverarbeitungsmaschine, keine Touchscreens) zur Ansteuerung von Theater-u. BΓü┐hnenscheinwerfer, Betriebsspannung: 100V-240V, S/N siehe Handelsrechnung PR21800615 / 69111000 ✗ (85371098)

## Plotting info
1. One plot is a square of classes, in every square is represented the
index of times is predicted one class when it's the other
It's good to be yellow on diagonal
2. The other is de function loss progression

In [0]:
if EXEC_MODE == 1 or EXEC_MODE == 2:
    plt.figure()
    plt.plot(all_losses)
if EXEC_MODE == 2:
    plt.figure()
    plt.plot(all_losses2)

In [0]:
test_with_plot(rnn, test_dataset, all_categories)
if EXEC_MODE == 2:
    test_with_plot(rnn2, test_dataset, all_categories)

## Saving and Loading
Saving and Loading model functions

In [0]:
save_checkpoint(rnn, PATH_MODEL_1)
if EXEC_MODE == 2:
    save_checkpoint(rnn2, PATH_MODEL_2)
print("saved")

### Some predicts

In [0]:
for key in view_dataset.keys():
    for value in view_dataset[key]:
        predict(rnn, value, all_categories, 3)